In [23]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
%matplotlib inline

def shuffle_in_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [62]:
import importlib
import VoxelDistanceFunctionFileLoader
importlib.reload(VoxelDistanceFunctionFileLoader)
from VoxelDistanceFunctionFileLoader import VoxelDistanceFunction
vdf = VoxelDistanceFunction("../data/test_output")
(examples_in, examples_out) = vdf.GenerateOccupancySamples(20, 2)

shuffle_in_unison(examples_in, examples_out)

N_training = 5000

examples_in_train = examples_in[:-N_training, :] 
examples_in_test = examples_in[-N_training:] 

examples_out_train = examples_out[:-N_training, :] 
examples_out_test = examples_out[-N_training:] 

# ffs
examples_in_test_positive = np.vstack([examples_in_test[x, :] for x in np.nonzero(examples_out_test)])

Generating 64000 examples...
			done.


In [42]:
# Check some statistics on the training set.
num_positive_examples = np.sum(examples_out_train)
num_negative_examples = examples_out_train.shape[0] - num_positive_examples
print(examples_in_train.shape, examples_out_train.shape)
print("Train: %d positive examples, %d negative examples" % (num_positive_examples, num_negative_examples))

num_positive_examples = np.sum(examples_out_test)
num_negative_examples = examples_out_test.shape[0] - num_positive_examples
print (examples_in_test.shape, examples_out_test.shape)
print("Test: %d positive examples, %d negative examples" % (num_positive_examples, num_negative_examples))

(59000, 8000) (59000, 1)
Train: 444 positive examples, 58556 negative examples
(5000, 8000) (5000, 1)
Test: 42 positive examples, 4958 negative examples


In [69]:
model = Sequential()

model.add(Dense(units=16, activation='relu', input_dim=examples_in.shape[1]))
model.add(Dense(units=1, activation='sigmoid'))

# For a binary classification problem
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
model.fit(examples_in_train, examples_out_train, epochs=5, batch_size=64)

Epoch 1/5
59000/59000 [==============================] - 3s - loss: 0.1447 - acc: 0.9933     
Epoch 2/5
59000/59000 [==============================] - 3s - loss: 0.0253 - acc: 0.9965     
Epoch 3/5
59000/59000 [==============================] - 3s - loss: 0.0213 - acc: 0.9974     
Epoch 4/5
59000/59000 [==============================] - 3s - loss: 0.0193 - acc: 0.9980     
Epoch 5/5
59000/59000 [==============================] - 3s - loss: 0.0183 - acc: 0.9982     


In [70]:
score = model.evaluate(examples_in_test, examples_out_test, batch_size=128)
print("\nScore: ", ["%s: %f" % (s, f) for (s, f) in zip(model.metrics_names, score)])

3840/5000 [======================>.......] - ETA: 0s
Score:  ['loss: 0.033830', 'acc: 0.996200']


In [71]:
examples_predictions_test_positive = model.predict_on_batch(examples_in_test_positive)
np.set_printoptions(suppress=True)
print(examples_predictions_test_positive)

[[ 0.99754924]
 [ 0.        ]
 [ 0.99992192]
 [ 0.        ]
 [ 0.99257857]
 [ 0.96478981]
 [ 0.00010617]
 [ 0.97644377]
 [ 0.99149394]
 [ 0.99187601]
 [ 0.95045638]
 [ 0.86626393]
 [ 0.9996165 ]
 [ 0.39521787]
 [ 0.        ]
 [ 0.99667192]
 [ 0.99807346]
 [ 0.97812498]
 [ 0.99961901]
 [ 0.        ]
 [ 0.98669398]
 [ 0.99996758]
 [ 0.9993999 ]
 [ 0.99998987]
 [ 0.94958013]
 [ 0.98219115]
 [ 0.9999938 ]
 [ 0.98890752]
 [ 0.99999511]
 [ 0.00000005]
 [ 0.99996316]
 [ 0.8606301 ]
 [ 0.99967444]
 [ 0.99826127]
 [ 0.02369842]
 [ 0.99896336]
 [ 0.99152124]
 [ 0.9992817 ]
 [ 0.09918093]
 [ 0.99963999]
 [ 0.        ]
 [ 0.99845767]
 [ 0.        ]
 [ 0.        ]
 [ 0.96104276]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000403]
 [ 0.00000

In [13]:
import ipyvolume
import scipy
v = vdf.occupancy
ipyvolume.quickvolshow(v, lighting=True, width=800, height=400, data_min=np.nanmin(v), data_max=np.nanmax(v))

/opt/conda/lib/python3.5/site-packages/ipyvolume/serialize.py:66: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)


A Jupyter Widget